# Goal
- Getting result statistics from saved MuscleDetector classes

# Dry run

In [1]:
%reset -f
%load_ext autoreload
%autoreload 2
#matplotlib figure size
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10
from custom_imports import *
from mri_project.metrics import *
from mri_project.utility import convert_list_col_to_multiple_cols

Matplotlib is building the font cache; this may take a moment.


In [2]:
files = glob.glob("../data/images_with_predictions_v04/*data")
len(files)

832

In [3]:
file_ = np.random.choice(files)
print(file_)
data_ = joblib.load(file_)

../data/images_with_predictions_v04/1-WF1B-4:50RepConRaw.data


In [4]:
data_.has_good_prediction()

True

In [5]:
data_.get_predicted_contours(89.8)
data_.get_traced_contours(89.8)
data_.get_contour_areas()
data_.get_contour_centers()

In [6]:
# fig, axes = plt.subplots(1, 2, figsize=(20, 10))
# axes[0].imshow(data_.traced_lever_arm_images[89.8])
# axes[1].imshow(data_.predicted_lever_arm_images[89.8])

In [7]:
# plt.imshow(data_.raw_image)

In [8]:
from collections import defaultdict
all_stats = defaultdict(dict)

In [9]:
a_ = np.array(data_.predicted_features['area'])
b_ = np.array(data_.traced_features['area'])
all_stats[data_.id]['area_ratio'] = dict(enumerate(a_ / b_))

In [10]:
a_ = np.array(data_.predicted_features['center'])
b_ = np.array(data_.traced_features['center'])

In [11]:
all_stats[data_.id]['center_distance'] = dict(enumerate(np.sqrt(np.sum((a_ - b_)**2, axis=1)) * data_.scale))

In [12]:
a_ = np.array(data_.predicted_features['lever_arm_89.8'])
b_ = np.array(data_.traced_features['lever_arm_89.8'])

In [13]:
all_stats[data_.id]['lever_arm_89.8_distance'] = dict(enumerate(a_ - b_))

In [14]:
a_ = data_.predicted
b_ = data_.traced_multilabel_mask

In [36]:
all_stats[data_.id]['total_accuracy'] = total_accuracy(a_, b_)
all_stats[data_.id]['accuracy'] = accuracy(a_, b_)
all_stats[data_.id]['precision'] = precision(a_, b_)
all_stats[data_.id]['recall'] = recall(a_, b_)

In [60]:
df_ = pd.DataFrame(all_stats).transpose()
df_.index = df_.index.map(lambda x: x.split("/")[-1])
df_ = df_.unstack()\
    .map(lambda x: list(x.items()) if isinstance(x, dict) else [(-1, x)])\
    .explode().to_frame()

In [68]:
convert_list_col_to_multiple_cols(df_, 0, ['muscle', 'value'])
# df_[['muscle', 'value']] = pd.DataFrame(df_[0].tolist(), index=df_.index)

In [70]:
df_.reset_index()

,level_0,level_1,muscle,value
0,accuracy,1-WF1B-4:50RepConRaw.jpg,0,0.897386
1,accuracy,1-WF1B-4:50RepConRaw.jpg,1,0.991863
2,accuracy,1-WF1B-4:50RepConRaw.jpg,2,0.795614
3,accuracy,1-WF1B-4:50RepConRaw.jpg,3,0.598726
4,accuracy,1-WF1B-4:50RepConRaw.jpg,4,0.965180
...,...,...,...,...
65,recall,1-WF1B-4:50RepConRaw.jpg,8,0.098938
66,recall,1-WF1B-4:50RepConRaw.jpg,9,0.006684
67,recall,1-WF1B-4:50RepConRaw.jpg,10,0.110259
68,recall,1-WF1B-4:50RepConRaw.jpg,11,0.385155


# Main run

In [114]:
%reset -f
%load_ext autoreload
%autoreload 2
#matplotlib figure size
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10
from custom_imports import *
from mri_project.metrics import *
from collections import defaultdict
from mri_project.utility import convert_list_col_to_multiple_cols

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [115]:
files = glob.glob("../data/images_with_predictions_v04/*data")
len(files)

832

11

In [246]:
pred_stats = {9: defaultdict(dict), 11: defaultdict(dict)}
bad_files = []
for index, file_ in enumerate(files):
    print(f"{index}: {file_}")
    data_ = joblib.load(file_)
    if not data_.has_good_prediction():
        print(f"bad prediction {file_}")
        bad_files.append(file_)
        continue
    data_.get_predicted_contours(89.8)
    data_.get_traced_contours(89.8)
    data_.get_contour_areas()
    data_.get_contour_centers()
    n_muscles = len(data_.predicted_contours)
    if n_muscles not in {9, 11}:
        bad_files.append(file_)
        continue
    a_ = np.array(data_.predicted_features['area'])
    b_ = np.array(data_.traced_features['area'])
    pred_stats[n_muscles][data_.id]['area_ratio'] = dict(enumerate(a_ / b_))
    a_ = np.array(data_.predicted_features['center'])
    b_ = np.array(data_.traced_features['center'])
# 
    pred_stats[n_muscles][data_.id]['center_distance'] = dict(enumerate(np.sqrt(np.sum((a_ - b_)**2, axis=1)) * data_.scale))
    a_ = np.array(data_.predicted_features['lever_arm_89.8'])
    b_ = np.array(data_.traced_features['lever_arm_89.8'])
# 
    pred_stats[n_muscles][data_.id]['lever_arm_89.8_distance'] = dict(enumerate(np.abs((a_ - b_))))
    a_ = data_.predicted
    b_ = data_.traced_multilabel_mask
    pred_stats[n_muscles][data_.id]['total_accuracy'] = total_accuracy(a_, b_)
    pred_stats[n_muscles][data_.id]['accuracy'] = accuracy(a_, b_)
    pred_stats[n_muscles][data_.id]['precision'] = precision(a_, b_)
    pred_stats[n_muscles][data_.id]['recall'] = recall(a_, b_)

0: ../data/images_with_predictions_v04/1-CO5-3:16RepPreRaw.data
1: ../data/images_with_predictions_v04/1-WF1B-4:50ConRaw.data
2: ../data/images_with_predictions_v04/2-WM8B-9:50ConRaw.data
3: ../data/images_with_predictions_v04/1-WF6B-2:16PostRaw.data
4: ../data/images_with_predictions_v04/1-CO2RR-5:50RepConRaw.data
5: ../data/images_with_predictions_v04/3-COM2B-11:16RepPostRaw.data
6: ../data/images_with_predictions_v04/1-COM5B-3:50RepConRaw.data
bad prediction ../data/images_with_predictions_v04/1-COM5B-3:50RepConRaw.data
7: ../data/images_with_predictions_v04/4-COF7B-21:50ConRaw.data
8: ../data/images_with_predictions_v04/4-COF5-14:16RepPreRaw.data
9: ../data/images_with_predictions_v04/1-WM3B-3:16PostRaw.data
10: ../data/images_with_predictions_v04/3-COF1-16:50RepConRaw.data
11: ../data/images_with_predictions_v04/3-COM6B-15:50ConRaw.data
12: ../data/images_with_predictions_v04/1-FCWF1-2:16RepPreRaw.data
13: ../data/images_with_predictions_v04/4-WF3B-14:16PostRaw.data
14: ../data/im

111: ../data/images_with_predictions_v04/2-FCWM1-7:16PreRaw.data
112: ../data/images_with_predictions_v04/4-COM1B-21:50ConRaw.data
113: ../data/images_with_predictions_v04/1-COF6-3:16PreRaw.data
114: ../data/images_with_predictions_v04/3-WM2B-15:50ConRaw.data
115: ../data/images_with_predictions_v04/2-FCCO6-7:16PreRaw.data
116: ../data/images_with_predictions_v04/3-COM2B-17:50ConRaw.data
117: ../data/images_with_predictions_v04/4-WF6B-13:16PostRaw.data
bad prediction ../data/images_with_predictions_v04/4-WF6B-13:16PostRaw.data
118: ../data/images_with_predictions_v04/1-CO4-1:50ConRaw.data
119: ../data/images_with_predictions_v04/1-COF7-6:50ConRaw.data
120: ../data/images_with_predictions_v04/3-WF6B-10:16PostRaw.data
121: ../data/images_with_predictions_v04/4-COM3B-21:50ConRaw.data
122: ../data/images_with_predictions_v04/2-COF1-10:50ConRaw.data
123: ../data/images_with_predictions_v04/1-WM1-4:50RepConRaw.data
124: ../data/images_with_predictions_v04/2-COM3B-9:50RepConRaw.data
125: ../d

muscles not of size 9 or 11
muscles not of size 9 or 11


183: ../data/images_with_predictions_v04/3-FCWF2-11:16PreRaw.data
184: ../data/images_with_predictions_v04/4-WM4B-22:50RepConRaw.data
185: ../data/images_with_predictions_v04/2-COF7-12:50ConRaw.data
186: ../data/images_with_predictions_v04/3-COF2B-11:16PostRaw.data
187: ../data/images_with_predictions_v04/2-CO5-6:16RepPreRaw.data
bad prediction ../data/images_with_predictions_v04/2-CO5-6:16RepPreRaw.data
188: ../data/images_with_predictions_v04/1-WM6B-2:16PostRaw.data
189: ../data/images_with_predictions_v04/3-CO2RR-11:16RepPreRaw.data
190: ../data/images_with_predictions_v04/1-FCWM2-3:16PreRaw.data
191: ../data/images_with_predictions_v04/1-COF6-5:50RepConRaw.data
192: ../data/images_with_predictions_v04/3-COM1B-11:16RepPostRaw.data
193: ../data/images_with_predictions_v04/4-WM2-20:50ConRaw.data
194: ../data/images_with_predictions_v04/2-WM9-4:50RepConRaw.data
195: ../data/images_with_predictions_v04/4-COF5B-15:16PostRaw.data
196: ../data/images_with_predictions_v04/2-WF6B-10:50RepCon

292: ../data/images_with_predictions_v04/3-WF6-11:16RepPreRaw.data
293: ../data/images_with_predictions_v04/2-FCWF1-6:16RepPreRaw.data
bad prediction ../data/images_with_predictions_v04/2-FCWF1-6:16RepPreRaw.data
294: ../data/images_with_predictions_v04/3-COF6-16:50RepConRaw.data
295: ../data/images_with_predictions_v04/3-COF7B-16:50RepConRaw.data
296: ../data/images_with_predictions_v04/1-WF1B-3:16RepPostRaw.data
297: ../data/images_with_predictions_v04/4-WF2B-15:16RepPostRaw.data
298: ../data/images_with_predictions_v04/2-FCWM6-7:16RepPreRaw.data
299: ../data/images_with_predictions_v04/1-COM4B-3:16PostRaw.data
bad prediction ../data/images_with_predictions_v04/1-COM4B-3:16PostRaw.data
300: ../data/images_with_predictions_v04/1-WM8B-5:50RepConRaw.data
301: ../data/images_with_predictions_v04/3-COF2B-16:50ConRaw.data
302: ../data/images_with_predictions_v04/4-FCWM2-14:16RepPreRaw.data
303: ../data/images_with_predictions_v04/1-REWF4-2:16PreRaw.data
304: ../data/images_with_predictions

...

538: ../data/images_with_predictions_v04/3-FCWF2-11:16RepPreRaw.data


muscles not of size 9 or 11


539: ../data/images_with_predictions_v04/2-COF3B-10:50RepConRaw.data
540: ../data/images_with_predictions_v04/2-WF6-11:50RepConRaw.data
541: ../data/images_with_predictions_v04/4-COF3B-21:50ConRaw.data
542: ../data/images_with_predictions_v04/3-COF6B-15:50ConRaw.data
543: ../data/images_with_predictions_v04/4-COM4B-15:16PostRaw.data
bad prediction ../data/images_with_predictions_v04/4-COM4B-15:16PostRaw.data
544: ../data/images_with_predictions_v04/2-WM2B-9:50ConRaw.data
545: ../data/images_with_predictions_v04/2-WM4B-10:50RepConRaw.data
bad prediction ../data/images_with_predictions_v04/2-WM4B-10:50RepConRaw.data
546: ../data/images_with_predictions_v04/2-WM3-9:50ConRaw.data
547: ../data/images_with_predictions_v04/2-WF5-9:50ConRaw.data
548: ../data/images_with_predictions_v04/4-WF4-18:50ConRaw.data
bad prediction ../data/images_with_predictions_v04/4-WF4-18:50ConRaw.data
549: ../data/images_with_predictions_v04/2-WM8-5:50ConRaw.data
550: ../data/images_with_predictions_v04/4-WF6-18:5

653: ../data/images_with_predictions_v04/2-WM6-5:50ConRaw.data
654: ../data/images_with_predictions_v04/2-WM3B-6:16RepPostRaw.data
655: ../data/images_with_predictions_v04/3-COF5-16:50ConRaw.data
656: ../data/images_with_predictions_v04/2-WF5B-7:16RepPostRaw.data
bad prediction ../data/images_with_predictions_v04/2-WF5B-7:16RepPostRaw.data
657: ../data/images_with_predictions_v04/3-CO4-12:50ConRaw.data
658: ../data/images_with_predictions_v04/4-WM6-17:50ConRaw.data
bad prediction ../data/images_with_predictions_v04/4-WM6-17:50ConRaw.data
659: ../data/images_with_predictions_v04/3-WM8B-16:50ConRaw.data
bad prediction ../data/images_with_predictions_v04/3-WM8B-16:50ConRaw.data
660: ../data/images_with_predictions_v04/1-WF3B-4:50RepConRaw.data
bad prediction ../data/images_with_predictions_v04/1-WF3B-4:50RepConRaw.data
661: ../data/images_with_predictions_v04/1-WM4-2:50RepConRaw.data
662: ../data/images_with_predictions_v04/1-COM6B-4:50ConRaw.data
663: ../data/images_with_predictions_v04/

750: ../data/images_with_predictions_v04/4-COM5B-15:16RepPostRaw.data
751: ../data/images_with_predictions_v04/2-COM2B-11:50ConRaw.data
752: ../data/images_with_predictions_v04/3-CO3-14:50RepConRaw.data
753: ../data/images_with_predictions_v04/4-COF7B-16:16RepPostRaw.data
754: ../data/images_with_predictions_v04/1-WM2B-3:16PostRaw.data
bad prediction ../data/images_with_predictions_v04/1-WM2B-3:16PostRaw.data
755: ../data/images_with_predictions_v04/4-COF7B-21:50RepConRaw.data
756: ../data/images_with_predictions_v04/4-COF4B-20:50ConRaw.data
757: ../data/images_with_predictions_v04/1-COF1-4:50ConRaw.data
bad prediction ../data/images_with_predictions_v04/1-COF1-4:50ConRaw.data
758: ../data/images_with_predictions_v04/3-COF7B-10:16PostRaw.data
759: ../data/images_with_predictions_v04/3-COF5-16:50RepConRaw.data
760: ../data/images_with_predictions_v04/4-WF4B-21:50RepPostRaw.data
761: ../data/images_with_predictions_v04/1-WM3B-3:16RepPostRaw.data
762: ../data/images_with_predictions_v04/2

In [223]:
len(bad_files)

1

In [234]:
df_1 = pd.DataFrame(pred_stats[9]).transpose()
# df_1.index = df_1.index.map(lambda x: x.split("/")[-1])
df_1 = df_1.unstack()\
    .map(lambda x: list(x.items()) if isinstance(x, dict) else [(-1, x)])\
    .explode().to_frame()

df_2 = pd.DataFrame(pred_stats[9]).transpose()
# df_2.index = df_2.index.map(lambda x: x.split("/")[-1])
df_2 = df_2.unstack()\
    .map(lambda x: list(x.items()) if isinstance(x, dict) else [(-1, x)])\
    .explode().to_frame()

In [235]:
convert_list_col_to_multiple_cols(df_1, 0, ['muscle', 'value'])
convert_list_col_to_multiple_cols(df_2, 0, ['muscle', 'value'])
df9 = df_1.reset_index().rename(columns={'level_0': 'metric', 'level_1': 'id'})
df11 = df_2.reset_index().rename(columns={'level_0': 'metric', 'level_1': 'id'})

In [240]:
df = pd.concat([df9.assign(n_muscles=9),  df11.assign(n_muscles=11)])

In [242]:
df[df.n_muscles==11].head()

,metric,id,muscle,value,n_muscles
0,area_ratio,../data/mri_analysis/2.Multigroup Set#2/Contro...,0,1.575099,11
1,area_ratio,../data/mri_analysis/2.Multigroup Set#2/Contro...,1,3.791567,11
2,area_ratio,../data/mri_analysis/2.Multigroup Set#2/Contro...,2,2.927293,11
3,area_ratio,../data/mri_analysis/2.Multigroup Set#2/Contro...,3,1.532780,11
4,area_ratio,../data/mri_analysis/2.Multigroup Set#2/Contro...,4,1.511721,11


In [184]:
df.groupby('id')['muscle'].max()

id
../data/mri_analysis/1.Multigroup Set#1_DONE/Control/Control - Post/Female/Subject F1B/Raw/1-COF1B-2:16PostRaw.jpg           11
../data/mri_analysis/1.Multigroup Set#1_DONE/Control/Control - Post/Female/Subject F1B/Raw/2-COF1B-7:16PostRaw.jpg           11
../data/mri_analysis/1.Multigroup Set#1_DONE/Control/Control - Post/Female/Subject F1B/Raw/3-COF1B-11:16PostRaw.jpg          11
../data/mri_analysis/1.Multigroup Set#1_DONE/Control/Control - Post/Female/Subject F1B/Raw/4-COF1B-15:16PostRaw.jpg          11
../data/mri_analysis/1.Multigroup Set#1_DONE/Control/Control - Post/Female/Subject F2B/Raw/1-COF2B-4:16PostRaw.jpg           11
                                                                                                                             ..
../data/mri_analysis/4.Contiguous Set#2/Weight Lifting/Weight Lifting - Pre/Male/Subject WM8/Raw/4-WM8-18:50RepConRaw.jpg    11
../data/mri_analysis/4.Contiguous Set#2/Weight Lifting/Weight Lifting - Pre/Male/Subject WM9/Raw/1-WM

In [158]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
df.reset_index()
res = df.groupby(['metric', 'muscle']).mean()# .reset_index().set_index('metric')
# res.loc['precision2'] = 1- res.loc['precision']
# res.loc['recall2'] = 1- res.loc['recall']

In [180]:
tmp_ = pd.get_dummies(res.reset_index(), columns=['muscle'])
tmp_.iloc[:, 2:] = tmp_.iloc[:, 2:].values * tmp_.iloc[:, 1].values.reshape(-1, 1)
tmp_.groupby('metric').sum()

,value,muscle_-1,muscle_0,muscle_1,muscle_2,muscle_3,muscle_4,muscle_5,muscle_6,muscle_7,muscle_8,muscle_9,muscle_10,muscle_11
metric,,,,,,,,,,,,,,
accuracy,11.171334,0.000000,0.922278,0.972075,0.881437,0.867833,0.884746,0.895648,0.973447,0.969967,0.957440,0.956661,0.948953,0.940849
area_ratio,19.605800,0.000000,1.404095,2.110597,2.191786,2.161709,2.079983,1.417634,1.375259,1.700688,1.744224,1.703574,1.716251,0.000000
center_distance,6.588037,0.000000,0.391722,0.700860,0.704428,0.722950,0.625412,0.478207,0.448997,0.529395,0.574163,0.667730,0.744174,0.000000
lever_arm_89.8_distance,4.286698,0.000000,0.008026,0.568789,0.576996,0.573871,0.485204,0.321401,0.314828,0.358092,0.351546,0.366455,0.361491,0.000000
precision,0.111662,0.000000,0.039029,0.004414,0.003174,0.003356,0.006903,0.006535,0.006693,0.006336,0.005141,0.004764,0.012482,0.012836
recall,0.828666,0.000000,0.077722,0.027925,0.118563,0.132167,0.115254,0.104352,0.026553,0.030033,0.042560,0.043339,0.051047,0.059151
total_accuracy,0.926223,0.926223,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [150]:
pd.reset_option("display.max_rows", "display.max_columns")

In [ ]:


df.groupby(['metric', 'muscle']).mean()